In [ ]:
%matplotlib inline
import numpy as np
from scipy import stats
from blimpy import Waterfall
from blimpy.utils import rebin
from matplotlib import pyplot as plt
from bisect import bisect_left
from tqdm import tqdm
import dask.array as da
import h5py
from time import time

from utils import *

fil_path = "data/filterbanks/"
h5_path = "data/h5/"

test_fil = fil_path + "blc20_guppi_57991_48899_3C161_0007.gpuspec.0000.fil"

fri_obs = h5_path + "GBT_57532_09539_HIP56445_fine.h5"

plt_args = {
            'aspect':'auto',
            'origin':'lower',
            'rasterized':True,
            'interpolation':'nearest',
            'cmap':'viridis'
            }
try:
    client.close()
except NameError:
    pass
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=3,
                n_workers=4, memory_limit='8GB')
client

In [ ]:
# Show Info
wf = Waterfall(fri_obs, load_data=False)
wf.info()

In [ ]:
channel_len = np.int(np.round(187.5/64/abs(wf.header[b'foff'])))
channel_len

In [ ]:
plt.figure()
wf.plot_waterfall(f_start=0, f_stop=1080)

In [ ]:
freqs, data = wf.grab_data(f_start=0, f_stop=1080)
print(data.shape)
freqs = freqs[::-1]
data = data[:,::-1]

In [ ]:
len(freqs)/channel_len

In [ ]:
data = data[:, :16 * 1033216]
freqs = freqs[:16 * 1033216]

integrated = np.sum(data, axis=0)/len(data)
for n in np.nonzero(integrated > 10**13):
    integrated[n] = (integrated[n-1] + integrated[n+1]) /2
plt.figure()
plt.plot(freqs, np.log(integrated))


In [ ]:
c = 6

window = data[:, channel_len*(c):channel_len*(c+1)]
plot_segment(np.log(window))

In [ ]:
window

In [ ]:
res = []
for chan in tqdm(range(15)):
    res.append([])
    window = data[:, channel_len*(chan):channel_len*(chan+1)]
    window_f = freqs[channel_len*(chan):channel_len*(chan+1)]
    for i in range(0, (len(window[0])//200*200), 100):
        test_data = window[:, i:i+200]
        s, p = norm_test(test_data)
        if p < 1e-25:
            res[chan].append((window_f[i], s, p))

In [ ]:
from multiprocessing import Pool
from time import time

import warnings
warnings.filterwarnings("ignore")


def threshold_hits(chan):
    res = list()
    window = data[:, channel_len*(chan):channel_len*(chan+1)]
    window_f = freqs[channel_len*(chan):channel_len*(chan+1)]
    for i in range(0, (len(window[0])//200*200), 100):
        test_data = window[:, i:i+200]
        s, p = norm_test(test_data)
        if p < 1e-25:
            res.append((window_f[i], s, p))
    return res

start = time()
with Pool(12) as p:
    chan_hits = p.map(threshold_hits, range(16))
end = time()
print(end-start)

In [ ]:
show_stamp_f(freqs, data, 1025.0100805927195)

In [ ]:
hits = [len(e) for e in chan_hits]
print(hits)
print(sum(hits))

In [ ]:
sorted_hits = sorted(res[7], key=lambda x: x[2])
sorted_hits

In [ ]:
top = [x[0] for x in sorted_hits[:20]]
top

In [ ]:
for i in top:
    print(i)
    show_stamp(window, i)

In [ ]:
res[:15]

In [ ]:
plt.figure()
plt.imshow(data[:, 8:8+128])

In [ ]:
with open("pfb512coef.txt", "r") as f:
    coef_file = f.read()

In [ ]:
lines = coef_file.splitlines()
filter_coefs = []
for line in lines:
    filter_coefs.append(float(line))

In [ ]:
coefs = np.array(filter_coefs)/2**17

In [ ]:
plt.figure()
plt.plot(coefs)

In [ ]:
from numpy import fft
l = 2**16
f = fft.fft(coefs, l)
plt.figure()
plt.plot(np.log(np.abs(f)**2))

In [ ]:
stacked = np.reshape(integrated, (15, 1033216))
plt.figure()
for i in range(15):
    plt.plot(np.log(stacked[i]))

In [ ]:
rebined = rebin(stacked, 1, 64)

In [ ]:
plt.figure()
plt.imshow(np.log(rebined), **plt_args)

In [ ]:
model_shape = np.sum(stacked, axis=0)/15
model_shape[model_shape > 10**10] = np.mean(model_shape)

In [ ]:
plt.figure()
plt.plot(model_shape)

In [ ]:
h5_file = h5py.File(fri_obs, "r")
a = da.from_array(h5_file["data"], chunks=(2, 1, channel_len*14))
a = a
a

In [ ]:
start = time()
a.persist()
da.to_npy_stack("original_a", a, axis=2)
end = time()
print(f"{end-start}")

In [ ]:
a = da.from_npy_stack("original_a")
a

In [ ]:
start = time()
means = da.mean(a, axis=2)
# means_data = means.compute()
# means_data
end = time()

print(f"{end-start}")
means = da.reshape(means, (16,1,1))
means

In [ ]:
a

In [ ]:
308/14

In [ ]:
normalized_a = da.divide(a, means)
normalized_a

In [ ]:
start = time()

da.to_npy_stack('normalized_a/', normalized_a, axis=2)

end = time()

print(f"{end-start}")

In [ ]:
print(normalized_a.shape)
print(normalized_a.chunks)

In [ ]:
start = time()
normalized_a_data = normalized_a.compute()
end = time()
print(f"{end-start}")

In [ ]:
start = time()

original_a_data = a[:, 0, :16 * 1033216].compute()

end = time()
print(f"{end-start}")

In [ ]:
read_a = da.from_array(original_a_data)
read_and_normalize = da.divide(read_a, means_data)

start = time()
normalized_a_data = read_and_normalize.compute()
end = time()
print(f"{end-start}")

# 1.6420118808746338

In [ ]:
del normalized_a_data

In [ ]:
start = time()

numpy_normalized_a_data = original_a_data / means_data

end = time()
print(f"{end-start}")

# 0.6659681797027588

In [ ]:
del numpy_normalized_a_data

In [ ]:
numpy_normalized_a_data.shape

In [ ]:
plot_segment(normalized_a_data)

In [ ]:
plot_segment(original_a_data)

In [ ]:
print(normalized_a_data.shape == numpy_normalized_a_data.shape)
print(np.allclose(normalized_a_data, numpy_normalized_a_data))